In [2]:
!nvidia-smi

Sat May  8 11:15:10 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.03   Driver Version: 450.119.03   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  On   | 00000000:3D:00.0 Off |                  N/A |
| 27%   26C    P8    20W / 250W |      3MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  On   | 00000000:3E:00.0 Off |                  N/A |
| 27%   

In [1]:
import argparse, os
import torch
import random
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from utils.vdsr import Net
from utils.dataset import DatasetFromHdf5

# Setting

training settings

In [2]:
batchSize = 128 # Training batch size
nEpochs = 50 # Number of epochs to train for
lr = 0.1 # Learning Rate
step = 10 # Sets the learning rate to the initial LR decayed by momentum every n epochs
resume = "" # Path to checkpoint
start_epoch = 1 # Manual epoch number (useful on restarts)
clip = 0.4 # Clipping Gradients
momentum = 0.9
weight_decay = 1e-4
pretrained = '' # path to pretrained model
checkpoint_step = 10 # the step to store checkpoints

GPU settings

In [3]:
cuda = True # use cuda
gpus = "8" # gpu ids

# Training

In [4]:
if cuda:
    print("=> use gpu id: '{}'".format(gpus))
    os.environ["CUDA_VISIBLE_DEVICES"] = gpus
    if not torch.cuda.is_available():
        raise Exception("No GPU found or Wrong gpu id, please run without --cuda")

=> use gpu id: '8'


In [5]:
seed = random.randint(1, 10000)
print("Random Seed: ", seed)
torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)

cudnn.benchmark = True

Random Seed:  6260


In [6]:
print("===> Loading datasets")
train_set = DatasetFromHdf5("data/train.h5")
training_data_loader = DataLoader(dataset=train_set, num_workers=1, batch_size=batchSize, shuffle=True)

===> Loading datasets


In [7]:
print("===> Building model")
model = Net()
criterion = nn.MSELoss(reduction='sum')

===> Building model


In [8]:
print("===> Setting GPU")
if cuda:
    model = model.cuda()
    criterion = criterion.cuda()

===> Setting GPU


In [9]:
# optionally resume from a checkpoint
if resume:
    if os.path.isfile(resume):
        print("=> loading checkpoint '{}'".format(resume))
        checkpoint = torch.load(resume)
        start_epoch = checkpoint["epoch"] + 1
        model.load_state_dict(checkpoint["model"].state_dict())
    else:
        print("=> no checkpoint found at '{}'".format(resume))

In [10]:
# optionally copy weights from a checkpoint
if pretrained:
    if os.path.isfile(pretrained):
        print("=> loading model '{}'".format(pretrained))
        weights = torch.load(pretrained)
        model.load_state_dict(weights['model'].state_dict())
    else:
        print("=> no model found at '{}'".format(pretrained))  

In [11]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 10 epochs"""
    update_lr = lr * (0.5 ** (epoch // step))
    return update_lr

def train(training_data_loader, optimizer, model, criterion, epoch):
    lr = adjust_learning_rate(optimizer, epoch-1)

    for param_group in optimizer.param_groups:
        param_group["lr"] = lr

    print("Epoch = {}, lr = {}".format(epoch, optimizer.param_groups[0]["lr"]))

    model.train()

    total_loss = 0
    cnt = 0
    for iteration, batch in enumerate(training_data_loader, 1):
        input, target = Variable(batch[0]), Variable(batch[1], requires_grad=False)

        if cuda:
            input = input.cuda()
            target = target.cuda()

        loss = criterion(model(input), target)
        optimizer.zero_grad()
        loss.backward() 
        nn.utils.clip_grad_norm(model.parameters(), clip) 
        optimizer.step()
        total_loss = total_loss + loss.data
        cnt = cnt + 1

    print("===> Epoch[{}]({}/{}): Loss: {:.10f}".format(epoch, iteration, len(training_data_loader), total_loss/cnt))

def save_checkpoint(model, epoch):
    model_out_path = "checkpoint/" + "model_epoch_{}.pth".format(epoch)
    state = {"epoch": epoch ,"model": model}
    if not os.path.exists("checkpoint/"):
        os.makedirs("checkpoint/")

    torch.save(state, model_out_path)

    print("Checkpoint saved to {}".format(model_out_path))

In [12]:
print("===> Setting Optimizer")
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

print("===> Training")
for epoch in range(start_epoch, nEpochs + 1):
    train(training_data_loader, optimizer, model, criterion, epoch)
    if epoch % checkpoint_step == 0:
        save_checkpoint(model, epoch)

===> Setting Optimizer
===> Training
Epoch = 1, lr = 0.1


/home/ichung/MTKlab_SR/sr/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


===> Epoch[1](1134/1134): Loss: 115.1066360474
Epoch = 2, lr = 0.1
===> Epoch[2](1134/1134): Loss: 68.5899963379
Epoch = 3, lr = 0.1
===> Epoch[3](1134/1134): Loss: 61.5759239197
Epoch = 4, lr = 0.1
===> Epoch[4](1134/1134): Loss: 57.9558868408
Epoch = 5, lr = 0.1
===> Epoch[5](1134/1134): Loss: 55.1494216919
Epoch = 6, lr = 0.1
===> Epoch[6](1134/1134): Loss: 53.4992904663
Epoch = 7, lr = 0.1
===> Epoch[7](1134/1134): Loss: 52.2871894836
Epoch = 8, lr = 0.1
===> Epoch[8](1134/1134): Loss: 51.1326217651
Epoch = 9, lr = 0.1
===> Epoch[9](1134/1134): Loss: 49.8642082214
Epoch = 10, lr = 0.1
===> Epoch[10](1134/1134): Loss: 48.8568458557
Checkpoint saved to checkpoint/model_epoch_10.pth
Epoch = 11, lr = 0.05
===> Epoch[11](1134/1134): Loss: 43.9031295776
Epoch = 12, lr = 0.05
===> Epoch[12](1134/1134): Loss: 42.6961097717
Epoch = 13, lr = 0.05
===> Epoch[13](1134/1134): Loss: 42.0108985901
Epoch = 14, lr = 0.05
===> Epoch[14](1134/1134): Loss: 41.2996292114
Epoch = 15, lr = 0.05
===> Epoc